<a href="https://colab.research.google.com/github/ShashankSatyam/HuggingFace---Transfer-Learning/blob/main/Hugging_Face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install datasets

# This will install the datasets library from the Python Package Index (PyPI).

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [ ]:
from datasets import load_dataset
multinews = load_dataset("multi_news" , split = 'test')

# # Load the 'test' split of the multi_news dataset

In [ ]:
multi_news = multinews.train_test_split(test_size = 0.2)

In [ ]:
multinews[0]

{'document': 'GOP Eyes Gains As Voters In 11 States Pick Governors \n \n Enlarge this image toggle caption Jim Cole/AP Jim Cole/AP \n \n Voters in 11 states will pick their governors tonight, and Republicans appear on track to increase their numbers by at least one, with the potential to extend their hold to more than two-thirds of the nation\'s top state offices. \n \n Eight of the gubernatorial seats up for grabs are now held by Democrats; three are in Republican hands. Republicans currently hold 29 governorships, Democrats have 20, and Rhode Island\'s Gov. Lincoln Chafee is an Independent. \n \n Polls and race analysts suggest that only three of tonight\'s contests are considered competitive, all in states where incumbent Democratic governors aren\'t running again: Montana, New Hampshire and Washington. \n \n While those state races remain too close to call, Republicans are expected to wrest the North Carolina governorship from Democratic control, and to easily win GOP-held seats in

In [ ]:
multinews

Dataset({
    features: ['document', 'summary'],
    num_rows: 5622
})

In [ ]:
pip install transformers accelerate

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")

#  Load the tokenizer for the "t5-small" model

In [ ]:
# Tokenizing a sample text
text = "Hello, how are you?"
tokens = tokenizer(text)

# to Print the tokenized output
print(tokens)

{'input_ids': [8774, 6, 149, 33, 25, 58, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}


In [ ]:
pre = 'sumarize'

def process(examples):
  inputs = [pre + doc for doc in examples['document']]

  model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

  labels = tokenizer(examples['summary'], max_length=128, truncation=True)

  model_inputs["labels"] = labels["input_ids"]

  return model_inputs

In [ ]:
tokenized_multi_news = multi_news.map(process, batched=True)

Map:   0%|          | 0/4497 [00:00<?, ? examples/s]

Map:   0%|          | 0/1125 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq , AutoModelForSeq2SeqLM , Seq2SeqTrainingArguments , Seq2SeqTrainer

data_collator = DataCollatorForSeq2Seq(tokenizer, model="t5-small")

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

""" DataCollatorForSeq2Seq: This class in the transformers library is used for preparing batches of data suitable for training seq2seq models.
                            It handles tasks like padding sequences to the maximum length within a batch.
                            -- data collection


    AutoModelForSeq2SeqLM:  This class is used to load a pre-trained seq2seq model from the Hugging Face model hub.
                             It supports automatic loading based on a model identifier, such as "t5-small".
                             AutoModelForSeq2SeqLM --  Loads a pre-trained seq2seq model for language modeling tasks.
                                    -- model loading

   DataCollatorForSeq2Seq:  Instantiates a DataCollatorForSeq2Seq object that will be used during training.
                              It takes a tokenizer and a model identifier as arguments.


           -- Seq2SeqTrainingArguments and Seq2SeqTrainer are further components used to configure and initiate the training process for a seq2seq model.

                                                 """

' DataCollatorForSeq2Seq: This class in the transformers library is used for preparing batches of data suitable for training seq2seq models. \n                            It handles tasks like padding sequences to the maximum length within a batch.  \n                            -- data collection \n                            \n                      \n    AutoModelForSeq2SeqLM:  This class is used to load a pre-trained seq2seq model from the Hugging Face model hub. \n                             It supports automatic loading based on a model identifier, such as "t5-small". \n                             AutoModelForSeq2SeqLM --  Loads a pre-trained seq2seq model for language modeling tasks.\n                                    -- model loading \n                             \n   DataCollatorForSeq2Seq:  Instantiates a DataCollatorForSeq2Seq object that will be used during training. \n                              It takes a tokenizer and a model identifier as arguments.    \n\n\n     

In [ ]:
training_arge = Seq2SeqTrainingArguments(
    output_dir = './results',
    evaluation_strategy = 'epoch',
    learning_rate = 2e-5,
    per_device_train_batch_size = 10,
    per_device_eval_batch_size = 10,
    weight_decay = 0.01,
    save_total_limit = 3,
    num_train_epochs = 10,
    fp16 = True,
)

# setting up training arguments.
 '''

    "The `evaluation_strategy` parameter is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead.",

   '''

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Seq2SeqTrainer(
    model = model,
    args = training_arge,
    train_dataset = tokenized_multi_news['train'],
    eval_dataset = tokenized_multi_news['test'],
    tokenizer = tokenizer,
    data_collator = data_collator,
)

# setting up trainer and initialising it.

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,2.875536
2,3.273700,2.813184
3,3.024000,2.782406
4,2.958200,2.762946
5,2.921500,2.752461
6,2.930000,2.745519
7,2.886500,2.738893
8,2.878900,2.735843
9,2.893200,2.734647
10,2.870900,2.732697


TrainOutput(global_step=4500, training_loss=2.959661349826389, metrics={'train_runtime': 2928.3797, 'train_samples_per_second': 15.357, 'train_steps_per_second': 1.537, 'total_flos': 1.217264162439168e+16, 'train_loss': 2.959661349826389, 'epoch': 10.0})

In [ ]:
'''training loss --  Measures how well the model fits the training data,
                     A decreasing training loss usually indicates better model performance on the training set.
                     Training loss is used to guide the optimization process during training.'''

In [ ]:
''' validation loss --- Measures how well the model fits new data, which is data that the model hasn't seen during training.
                        It can also help prevent overfitting.    '''